In [1]:
import glob, json, os
import numpy as np
from xml.dom import minidom

In [2]:
def convertTransformationTemplate(data):
    transform_template = dict()

    transform_template['fl_x'] = np.float(data.getElementsByTagName('f')[0].childNodes[0].data)
    transform_template['fl_y'] = transform_template['fl_x']
    transform_template['cx'] = np.float(data.getElementsByTagName('cx')[0].childNodes[0].data)
    transform_template['cy'] = np.float(data.getElementsByTagName('cy')[0].childNodes[0].data)
    
    transform_template['camera_angle_x'] = np.arctan(transform_template['cx']/transform_template['fl_x'])*2
    transform_template['camera_angle_y'] = np.arctan(transform_template['cy']/transform_template['fl_y'])*2
    
    dimension = data.getElementsByTagName('resolution')[0]
    transform_template['w'] = np.float(dimension.attributes['width'].value)
    transform_template['h'] = np.float(dimension.attributes['height'].value)
    
    cameras = xml_data.getElementsByTagName('transform')
    
    def convertFrameTemplate(cameras, n):
        frame_template = dict()
        frame_template['file_path'] = str(n+1) + '_masked'
        frame_template['transform_matrix'] = np.array(cameras[n].childNodes[0].data.split(' '), dtype=np.float).reshape((4,4)).tolist()
        return frame_template
    
    transform_template['frames'] = [convertFrameTemplate(cameras, 0)]
    for i in range(1, len(cameras)):
        transform_template['frames'].append(convertFrameTemplate(cameras, i))
    return transform_template

In [3]:
xml_paths = glob.glob('/home/leyinghu/Documents/data/Real_data07182022/Cams_calib.xml')
xml_paths.sort()

In [4]:
for xml_path in xml_paths:
    
    for trainOrTest in ['train', 'test']:
        xml_dir = os.path.dirname(xml_path)
        transforms_path = os.path.join(xml_dir, 'transforms_' + trainOrTest + '.json')
        if os.path.exists(transforms_path):
            os.remove(transforms_path)

        xml_data = minidom.parse(xml_path)
        images = glob.glob(os.path.join(xml_dir, 'diffuse', '*_diffuse.png'))
        
        cameras = xml_data.getElementsByTagName('transform')
        assert len(cameras) == len(images)

        transformations_template = convertTransformationTemplate(xml_data)

        with open(transforms_path, 'w') as f:
            json.dump(transformations_template, f, indent=2)
        

/tmp/ipykernel_4772/3097377641.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  transform_template['fl_x'] = np.float(data.getElementsByTagName('f')[0].childNodes[0].data)
/tmp/ipykernel_4772/3097377641.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  transform_template['cx'] = np.float(data.getElementsByTagName('cx')[0].childNodes[